In [2]:
import numpy as np
import cv2
import pickle
import statistics as stat

In [5]:
caspath = 'C:/Users/Andrija/.spyder-py3/cascades/data/haarcascade_frontalface_alt2.xml'
face_cascade = cv2.CascadeClassifier(caspath)
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("C:/Users/Andrija/.spyder-py3/trainner.yml")
path = 'C:/Users/Andrija/.spyder-py3/images/new/'
labels = {}
with open("C:/Users/Andrija/.spyder-py3/labels.pickle", 'rb') as f:
    og_lables = pickle.load(f)
    labels = {v:k for k,v in og_lables.items()} # to invert lables 

cap = cv2.VideoCapture(0)
stor_conf = []
while True:
    key = cv2.waitKey(20) & 0xFF
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.5,minNeighbors = 5 )
    for (x, y, w, h) in faces:
        #print (x, y , w, h)
        roi_gray = gray[y:y+h, x:x+w] # roi is region of interst
        roi_color = frame[y:y+h, x:x+w]
        canny = cv2.Canny(frame,100,100)
        id_, conf = recognizer.predict(roi_gray)
        
        
     
            
        if conf >= 60 and id_ == 0:
            print(id_)           
            print(labels[id_], conf)
            stor_conf.append(conf)
            
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color = (255,255,255)
            stroke = 2 
            
            cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)
        if id_ != 0 or conf < 60:
            font = cv2.FONT_HERSHEY_SIMPLEX
            color = (255,255,255)
            stroke = 2 
            cv2.putText(frame, "Unknown", (x,y), font, 1, color, stroke, cv2.LINE_AA)
        
            
            
        color = (255,0,0)
        stroke = 2
        end_cord_x = x+w #width
        end_cord_y = y+h #height
        cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, stroke)
    
    cv2.imshow('frame', frame)
    if len(stor_conf) > 10 and stat.mean(stor_conf) > 70:
        break
    
    if  key == ord('q'):
        break
    
     
cap.release()
cv2.destroyAllWindows()